In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/통합데이터/VIP데이터.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11460 entries, 0 to 11459
Columns: 898 entries, 기준년월 to 최종카드발급경과월
dtypes: float64(62), int64(798), object(38)
memory usage: 78.5+ MB


In [25]:
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# df 경로는 각자 설정
df = pd.read_csv("../../data/통합데이터/Final_merged_all_data.csv")
# df = df.sort_values(by=['발급회원번호', '기준년월'])

# -------------------------------------------------------
# 1. 6개월 기울기(Slope) 계산 (df에 바로 생성)
# -------------------------------------------------------
def calc_slope_6m(series):
    if len(series) < 2 or np.sum(series) == 0: return 0
    y = series.values
    x = np.arange(len(y))
    slope, _, _, _, _ = linregress(x, y)
    return slope

# transform으로 전체 데이터에 기울기 적용
df['Slope_Spend'] = df.groupby('발급회원번호')['이용금액_신용_B0M'].transform(calc_slope_6m)
df['Slope_Balance'] = df.groupby('발급회원번호')['잔액_B0M'].transform(calc_slope_6m)
df['Slope_Count'] = df.groupby('발급회원번호')['이용건수_신용_B0M'].transform(calc_slope_6m)

# -------------------------------------------------------
# 2. 현재 시점(12월) 데이터만 추출
# -------------------------------------------------------
target_month = df['기준년월'].max()
df_final = df[df['기준년월'] == target_month].copy()

# -------------------------------------------------------
# 3. [핵심] 기울기를 '이탈 점수'로 변환 (Normalization)
# -------------------------------------------------------
# 전략: 기울기가 음수일수록(가파른 감소) 높은 점수를 줍니다.
# 양수(증가)인 경우는 이탈 위험이 없으므로 0점으로 처리합니다.

scaler = MinMaxScaler()

def convert_slope_to_score(col_name):
    # 1. 감소하는 경우(음수)만 추출, 나머지는 0으로 대치
    # (음수 값에 -1을 곱해 양수로 만듦: -5000 -> 5000)
    # 즉, 값이 클수록 '많이 줄어듦'을 의미하게 됨
    negative_slopes = df_final[col_name].apply(lambda x: -x if x < 0 else 0)
    
    # 2. 0~1 사이 값으로 정규화 (MinMax Scaling)
    # 가장 많이 줄어든 사람이 1.0, 안 줄어든 사람이 0.0
    return scaler.fit_transform(negative_slopes.values.reshape(-1, 1)).flatten()

# 각 지표별 '감소 심각도' (0.0 ~ 1.0)
score_spend_norm = convert_slope_to_score('Slope_Spend')
score_balance_norm = convert_slope_to_score('Slope_Balance')
score_count_norm = convert_slope_to_score('Slope_Count')

# -------------------------------------------------------
# 4. 가중치 적용 및 최종 점수 산출 (100점 만점)
# -------------------------------------------------------
# 가중치 설정 (비즈니스 중요도에 따라 조절 가능)
W_SPEND = 40   # 소비 감소 비중
W_BALANCE = 30 # 잔액 감소(이탈준비) 비중
W_COUNT = 20   # 빈도 감소 비중
W_RISK = 10    # 현재 연체 등 리스크 비중

# (1) 리스크 점수 별도 계산 (0 or 10점)
# 연체나 승인거절이 하나라도 있으면 10점 만점 부여
has_risk = np.where(
    (df_final.get('연체잔액_B0M', 0) > 0) | 
    (df_final.get('승인거절건수_B0M', 0) > 0), 
    1, 0
)

# (2) 최종 스코어 계산
df_final['Churn_Score'] = (
    (score_spend_norm * W_SPEND) + 
    (score_balance_norm * W_BALANCE) + 
    (score_count_norm * W_COUNT) + 
    (has_risk * W_RISK)
)

# 소수점 정리
df_final['Churn_Score'] = df_final['Churn_Score'].round(1)

# 출력
print("\n[이탈 위험 점수(Churn Score) 상위 10명]")
cols_view = ['발급회원번호', 'Slope_Spend', 'Slope_Balance', 'Churn_Score']
print(df_final[cols_view].sort_values('Churn_Score', ascending=False).head(10))



[이탈 위험 점수(Churn Score) 상위 10명]
         발급회원번호    Slope_Spend  Slope_Balance  Churn_Score
56996  SYN_6996 -259949.057143 -466081.371429         65.5
52794  SYN_2794 -252282.000000 -115305.971429         59.9
53968  SYN_3968 -243906.428571 -117226.657143         51.1
59037  SYN_9037 -229686.857143 -119831.371429         47.7
55559  SYN_5559 -269313.542857  -44089.685714         45.9
53956  SYN_3956 -222279.228571 -168815.114286         42.1
55455  SYN_5455 -223850.714286  -61721.000000         41.3
55296  SYN_5296 -206826.314286 -381888.542857         41.0
53442  SYN_3442 -168908.285714 -186469.485714         39.1
51527  SYN_1527 -186951.657143 -241112.200000         38.0


In [27]:
df_final.columns

Index(['기준년월', '발급회원번호', '컨택건수_카드론_TM_B0M', '컨택건수_리볼빙_TM_B0M',
       '컨택건수_CA_TM_B0M', '컨택건수_이용유도_TM_B0M', '컨택건수_신용발급_TM_B0M',
       '컨택건수_부대서비스_TM_B0M', '컨택건수_포인트소진_TM_B0M', '컨택건수_보험_TM_B0M',
       ...
       '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', '카드신청건수', 'Life_Stage',
       '최종카드발급경과월', 'Slope_Spend', 'Slope_Balance', 'Slope_Count',
       'Churn_Score'],
      dtype='object', length=902)

In [28]:
cols = pd.read_csv('../../data/Selected_Top150_Features.csv')

In [29]:
features = cols['변수명'].to_list()

In [30]:
features.append('발급회원번호')
features.append('기준년월')
features.append('Churn_Score')
features

['증감율_이용건수_일시불_분기',
 '납부_기타이용금액',
 '승인거절건수_B0M',
 '납부_보험료이용금액',
 '납부_통신비이용금액',
 '증감율_이용건수_신용_분기',
 '최종카드발급일자',
 '쇼핑_도소매_이용금액',
 '_2순위교통업종_이용금액',
 '잔액_할부_유이자_B0M',
 '증감율_이용건수_CA_분기',
 '연속유실적개월수_기본_24M_카드',
 '변동률_잔액_B1M',
 '변동률_RV일시불평잔',
 '상담건수_R6M',
 '변동률_일시불평잔',
 '이용금액_요식',
 '쇼핑_아울렛_이용금액',
 '이용메뉴건수_IB_R6M',
 'IB문의건수_선결제_R6M',
 '연체건수_R6M',
 '이용금액_체크_R12M',
 '유효카드수_체크',
 '증감율_이용금액_일시불_전월',
 '증감율_이용건수_일시불_전월',
 '최대이용금액_할부_무이자_R12M',
 '할인금액_청구서_B0M',
 '최종이용일자_기본',
 '쇼핑_편의점_이용금액',
 '이용건수_할부_유이자_R12M',
 '최종이용일자_카드론',
 '혜택수혜율_B0M',
 '월상환론한도금액',
 '이용금액_해외',
 '최종카드론_거치개월수',
 '선입금원금_B5M',
 '이용금액_오프라인_B0M',
 '컨택건수_카드론_TM_R6M',
 '_2순위업종_이용금액',
 '증감율_이용금액_신용_전월',
 '연체입금원금_B2M',
 '교통_버스지하철이용금액',
 '최대이용금액_일시불_R12M',
 '입회일자_신용',
 '증감율_이용금액_신판_분기',
 '포인트_적립포인트_R12M',
 '이용후경과월_신용',
 '대표결제일',
 '_1순위교통업종_이용금액',
 '일시상환론한도금액',
 '정상입금원금_B5M',
 '최종이용일자_CA',
 '할부금액_무이자_3M_R12M',
 '_1순위쇼핑업종_이용금액',
 '이용금액_일상생활',
 '카드이용한도금액',
 '캠페인접촉일수_R12M',
 '연체입금원금_B5M',
 '잔액_신판최대한도소진율_r6m',
 '상향가능한도금액',
 '할부금액_12M_R12M',
 '포인

In [31]:
df_final[features]

,증감율_이용건수_일시불_분기,납부_기타이용금액,승인거절건수_B0M,납부_보험료이용금액,납부_통신비이용금액,증감율_이용건수_신용_분기,최종카드발급일자,쇼핑_도소매_이용금액,_2순위교통업종_이용금액,잔액_할부_유이자_B0M,...,불만제기후경과월_R12M,컨택건수_이용유도_인터넷_B0M,이용금액_D페이_R6M,이용카드수_체크,이용금액_B페이_R6M,이용개월수_C페이_R6M,이용금액_페이_오프라인_B0M,발급회원번호,기준년월,Churn_Score
50000,-0.260578,4500,0,197020,0,-0.260578,20160912.0,0,0,0,...,12,0,0,0,0,0,0,SYN_0,201812,3.3
50001,-0.288179,0,0,0,226049,-0.288179,20170122.0,60907,0,149135,...,12,0,0,0,0,0,0,SYN_1,201812,4.8
50002,-0.162731,0,0,0,0,-0.162731,20180227.0,0,0,0,...,0,0,0,0,0,0,0,SYN_2,201812,0.0
50003,-0.043021,0,0,0,0,-0.043021,20141231.0,0,0,0,...,0,0,0,0,0,0,0,SYN_3,201812,0.3
50004,0.000000,0,0,0,0,0.000000,20170206.0,0,0,0,...,0,0,0,0,0,0,0,SYN_4,201812,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,-0.423876,0,0,0,263700,-0.423876,20180715.0,0,0,0,...,12,1,0,0,0,4,0,SYN_9995,201812,5.9
59996,0.000000,0,0,0,0,0.000000,NaN,0,0,0,...,0,0,0,0,0,0,0,SYN_9996,201812,0.0
59997,0.136125,0,0,0,170593,0.027790,20171125.0,179852,0,321085,...,12,0,0,1,414040,0,178270,SYN_9997,201812,14.0
59998,0.000000,1937,0,0,0,0.000000,20180616.0,0,0,0,...,12,1,0,0,0,0,0,SYN_9998,201812,7.0
